In [1]:
import torch
def make_anchors(feats, strides, grid_cell_offset=0.5):
    """Generate anchors from features."""
    anchor_points, stride_tensor = [], []
    assert feats is not None
    dtype, device = feats[0].dtype, feats[0].device
    for i, stride in enumerate(strides):
        _, _, h, w = feats[i].shape
        sx = torch.arange(end=w, device=device, dtype=dtype) + grid_cell_offset  # shift x
        sy = torch.arange(end=h, device=device, dtype=dtype) + grid_cell_offset  # shift y
        sy, sx = torch.meshgrid(sy, sx)
        anchor_points.append(torch.stack((sx, sy), -1).view(-1, 2))
        stride_tensor.append(torch.full((h * w, 1), stride, dtype=dtype, device=device))
    return torch.cat(anchor_points), torch.cat(stride_tensor)

/home/uih/anaconda3/envs/torch1.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
feats = [torch.rand(4, 65, 160, 160), torch.rand(4, 65, 80, 80), torch.rand(4, 65, 40, 40), torch.rand(4, 65, 20, 20)]
strides = [4, 8, 16, 32]
make_anchors(feats, strides)

(tensor([[ 0.5000,  0.5000],
         [ 1.5000,  0.5000],
         [ 2.5000,  0.5000],
         ...,
         [17.5000, 19.5000],
         [18.5000, 19.5000],
         [19.5000, 19.5000]]),
 tensor([[ 4.],
         [ 4.],
         [ 4.],
         ...,
         [32.],
         [32.],
         [32.]]))